<h1>
<center>
Module 8: Artifical Neural Nets Assignment
</center>
</h1>
<div class=h1_cell>
<p>
I think it would be fun to try something out. Let's apply our ANN method to predicting loans. We have been using what are termed "shallow" machine learning methods to this point. Let's see what we can do with an ANN.
</div>

In [1]:
import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
with open('/content/gdrive/My Drive/class_tables/loan_table_week4.csv', 'r') as f:
  loan_table = pd.read_csv(f)

In [0]:
!rm library_w19_deep_1.py

In [4]:
from google.colab import files
files.upload()

Saving library_w19_deep_1.py to library_w19_deep_1.py


{'library_w19_deep_1.py': b"import pandas as pd\nimport numpy as np\n\ndef sigmoid(x):  \n    return 1/(1+np.exp(-x))\n\ndef mse(z,y):\n  return (z-y)**2\n\ndef mse_der(z,y):\n  return z-y\n\ndef sigmoid_der(x):  \n    return sigmoid(x)*(1-sigmoid(x))\n\ndef ann_simple(all_samples, labels, weights, bias, hypers={}):\n  \n  '''\n  Can build an ANN with n input nodes and one output node.\n  Uses sigmoid and mse.\n  '''\n  \n  input_n = all_samples.shape[1]  #number of inputs in each sample\n  \n  assert weights.shape == (input_n,1), 'weights needs to have same shape as sample'\n  assert all_samples.shape[0] >= 1, 'all_samples must represent 1 or more samples'\n  assert bias.shape == (1,) , 'a single bias weight for output node'\n  assert labels.shape[1] == 1, 'actual value for the 1 output node'\n  assert labels.shape[0] == all_samples.shape[0], 'labels must match up with samples'\n  \n  hyper_keys = [*hypers]  #fails on 2.7\n  target_set = set(['epochs', 'cost-reporting', 'learning-rate

In [5]:
from library_w19_deep_1 import *

%who function

ann_predictor	 ann_simple	 ann_tester	 from_scratch	 mse	 mse_der	 sigmoid	 sigmoid_der	 


In [0]:
import numpy as np

<h2>Challenge 1: connect Loan Table to simple ANN (30 points)</h2>

Choose 3 columns from loan_table and wrangle them into shape so we can feed our simple ANN.

<h2>Reminder of what we are working with</h2>

I'm saying we can choose 3 columns from the loan table and use the values for the 3 inputs x1,x2,x3.

<img src='https://www.dropbox.com/s/09vo4h9mufmo6tn/Screenshot%202019-02-01%2015.23.42.png?raw=1'>

<h2>Choice of our 3 columns</h2>

Here are the columns we have to choose from.

In [7]:
loan_table.head(1)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,...,apin_binned,apin_low,apin_average,apin_high,apin_nan,dep_0,dep_1,dep_2,dep_3+,dep_nan
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,...,low,1,0,0,0,1,0,0,0,0


In [8]:
loan_table.describe(include='all')

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,...,apin_binned,apin_low,apin_average,apin_high,apin_nan,dep_0,dep_1,dep_2,dep_3+,dep_nan
count,601,611,599,614,582,614.000000,614.000000,592.000000,600.00000,564.000000,...,614,614.000000,614.000000,614.000000,614.0,614.000000,614.000000,614.000000,614.000000,614.000000
unique,2,2,4,2,2,NaN,NaN,NaN,NaN,NaN,...,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,Male,Yes,0,Graduate,No,NaN,NaN,NaN,NaN,NaN,...,low,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,489,398,345,480,500,NaN,NaN,NaN,NaN,NaN,...,607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,5403.459283,1621.245798,146.412162,342.00000,0.842199,...,NaN,0.988599,0.008143,0.003257,0.0,0.561889,0.166124,0.164495,0.083062,0.024430
std,NaN,NaN,NaN,NaN,NaN,6109.041673,2926.248369,85.587325,65.12041,0.364878,...,NaN,0.106250,0.089945,0.057026,0.0,0.496559,0.372495,0.371027,0.276201,0.154506
min,NaN,NaN,NaN,NaN,NaN,150.000000,0.000000,9.000000,12.00000,0.000000,...,NaN,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,2877.500000,0.000000,100.000000,360.00000,1.000000,...,NaN,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
50%,NaN,NaN,NaN,NaN,NaN,3812.500000,1188.500000,128.000000,360.00000,1.000000,...,NaN,1.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000
75%,NaN,NaN,NaN,NaN,NaN,5795.000000,2297.250000,168.000000,360.00000,1.000000,...,NaN,1.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000


I like `Credit_History` in its raw form - not ohe form. No need to use ohe with the ANN.
<p>
  I like `ApplicantIncome` in its raw form. Again, no need to bin or ohe this column. In essence, we are asking the ANN to find the bins that are most useful. You will have to normalize this column.
  <p>
    I like `Married` but we will have to transform into 0 and 1.
    <p>
      I am not going to mess around with rows that have an empty value in one of the 3 columns. I am just going to throw them out.
      <p>
        

<h2>My way</h2>

I'll give you the steps I followed. However, you don't have to follow these steps. If you can get to the goal some other way, please feel free. What is our goal? To have the variables `feature_set` and `labels` set up so we can call `from_scratch` as shown below.

<img src='https://www.dropbox.com/s/h6vjn99ybtsozjn/Screenshot%202019-02-19%2008.45.31.png?raw=1'><br>
<img src='https://www.dropbox.com/s/7irwbe327c4tv8y/Screenshot%202019-02-19%2008.46.02.png?raw=1'><br>
<img src='https://www.dropbox.com/s/wokpipqobpkg83a/Screenshot%202019-02-19%2008.46.27.png?raw=1'>
<p>
  And here are my results:
  <p>
    <img src='https://www.dropbox.com/s/uvs3ni7vwa3s2ai/Screenshot%202019-02-24%2010.24.03.png?raw=1'>

<h3>My way: step 1</h3>

Build a new table that only has the 3 columns plus the target column.

In [9]:

raw_table.head(1)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,...,apin_binned,apin_low,apin_average,apin_high,apin_nan,dep_0,dep_1,dep_2,dep_3+,dep_nan
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,...,low,1,0,0,0,1,0,0,0,0


In [10]:

ann_table.head(1)

,Credit_History,ApplicantIncome,Married,Loan_Status
0,1.0,5849,No,1


<h3>My way: step 2</h3>

Drop all rows with a NaN.

In [12]:
ann_table.describe(include='all')

,Credit_History,ApplicantIncome,Married,Loan_Status
count,561.000000,561.000000,561,561.000000
unique,NaN,NaN,2,NaN
top,NaN,NaN,Yes,NaN
freq,NaN,NaN,362,NaN
mean,0.841355,5462.475936,NaN,0.680927
std,0.365671,6284.680488,NaN,0.466533
min,0.000000,150.000000,NaN,0.000000
25%,1.000000,2889.000000,NaN,0.000000
50%,1.000000,3813.000000,NaN,1.000000
75%,1.000000,5800.000000,NaN,1.000000


<h3>My way: step 3</h3>

Wrangle the Married column into binary form.

In [14]:
ann_table.head(1)

,Credit_History,ApplicantIncome,Married,Loan_Status
0,1.0,5849,0,1


<h3>My way: step 4</h3>

Normalize the ApplicantIncome column.


In [17]:
ann_table.head()

,Credit_History,Married,Loan_Status,apin_normed
0,1.0,0,1,0.072210
1,1.0,1,0,0.056580
2,1.0,1,1,0.037037
3,1.0,1,1,0.031889
4,1.0,0,1,0.074074


<h3>Most of way to having feature set</h3>

At this point I have the following table.<br>
<img src='https://www.dropbox.com/s/4yq0hbputlh8gt9/Screenshot%202019-02-19%2008.58.12.png?raw=1'>

<h3>My way: step 5</h3>

Turning attention to labels. Pull them out of table and get them into numpy array of correct shape, i.e., (561, 1).

In [19]:

print(labels.shape)

(561, 1)


In [21]:
ann_table.head()

,Credit_History,Married,apin_normed
0,1.0,0,0.072210
1,1.0,1,0.056580
2,1.0,1,0.037037
3,1.0,1,0.031889
4,1.0,0,0.074074


<h3>My way: step 6</h3>

After dropping the Loan_Status column, all I need to do is turn my table into a list of lists. And then turn that into a numpy array.

In [23]:

print(feature_set.shape)
feature_set[0]

(561, 3)


array([1.        , 0.        , 0.07220988])

<h3>I am good to go</h3>

I have `feature_set` and `labels` set up. Ready to call `from_scratch`.

In [24]:
from_scratch(feature_set, labels, hypers={'epochs':10000, 'cost-reporting':1000, 'learning-rate': .05}) #can take awhile

(0, array([0.20333739]))
(1000, array([0.15038711]))
(2000, array([0.15023848]))
(3000, array([0.1502394]))
(4000, array([0.15023943]))
(5000, array([0.15023943]))
(6000, array([0.15023943]))
(7000, array([0.15023943]))
(8000, array([0.15023943]))
(9000, array([0.15023943]))
(10000, array([0.15023943]))


(array([[ 4.16472838],
        [ 0.57575141],
        [-0.74925534]]), array([-2.88134978]))

In [25]:
#copied and pasted from above results

ann_tester(feature_set, labels, [[ 4.16472838], [ 0.57575141], [-0.74925534]], [-2.88134978])

0.8146167557932263

<h2>Not bad</h2>

With just 4 weights we are able to significantly improve over the accuracy we got from random forests.

<h2>Can we do better by adjusting the learning rate?</h2>
​
Let's see if we can do better by decreasing the learning rate. We might be in a local minima/valley and can crawl out with smaller steps.


In [26]:
from_scratch(feature_set, labels, hypers={'epochs':10000, 'cost-reporting':1000, 'learning-rate': .01}) #can take awhile

(0, array([0.21049884]))
(1000, array([0.15100094]))
(2000, array([0.1501018]))
(3000, array([0.15008394]))
(4000, array([0.15008279]))
(5000, array([0.15008498]))
(6000, array([0.15008686]))
(7000, array([0.150088]))
(8000, array([0.15008861]))
(9000, array([0.15008892]))
(10000, array([0.15008908]))


(array([[ 4.14912334],
        [ 0.56890032],
        [-0.74241646]]), array([-2.87084687]))

In [27]:
#copied and pasted from above results

ann_tester(feature_set, labels, [[ 4.14912334], [ 0.56890032], [-0.74241646]], [-2.87084687])

0.8146167557932263

No help. Same as with learning rate of `.05`. I could continue to play with the learning rate but am going to switch to another strategy: add another feature.

<h1>Challenge 2: add a 4th column (20 points)</h1>

Build a feature set that has 4 values taken from ['Credit_History', 'ApplicantIncome', 'Married', 'Education']. You will have to wrangle `Education`.
<p>
Build your ANN using from_scratch and these hypers: hypers={'epochs':10000, 'cost-reporting':1000, 'learning-rate': .05}.
  <p>
    Test it with ann_tester. 

In [32]:
fs.shape

(561, 4)

In [33]:
lab.shape

(561, 1)

In [34]:
from_scratch(fs, lab, hypers={'epochs':10000, 'cost-reporting':1000, 'learning-rate': .05}) #can take awhile

(0, array([0.2030543]))
(1000, array([0.15007264]))
(2000, array([0.14992259]))
(3000, array([0.14992345]))
(4000, array([0.14992347]))
(5000, array([0.14992347]))
(6000, array([0.14992347]))
(7000, array([0.14992347]))
(8000, array([0.14992347]))
(9000, array([0.14992347]))
(10000, array([0.14992347]))


(array([[ 4.11963466],
        [ 0.55316336],
        [ 0.34577307],
        [-0.96094329]]), array([-3.09416354]))

Looks like our ending cost is about the same than with just 3 columns. That should lead to roughly same accuracy than with 3 columns, i.e., `0.8146`. Let's see.

In [35]:
ann_tester(fs, lab, [[ 4.11293303], [-0.5402908 ], [ 0.34635578], [-0.96150273]], [-2.53688187])

0.8146167557932263

The same.
<p>
  From here we could try different combos of columns and also adjust the hyper-parameters. This is exactly where most of the time is spent in deep learning. It is easy to build the models but hard to find really good parameter settings.

<h1>Challenge 3: Batch it (50 points)</h1>

I have mentioned the term "batch" in several places but never really defined it. Batch has to do with the slice of samples we take before updating weights. We have been using a batch size of 1; we take 1 sample and then do backprop and update the weights. This is also referred to as online learning or pure stochastic gradient descent.
<p>
  What are other options? We could delay updating weights until after we have seen n samples. If n is the length of entire feature set, it is called batch learning. If n is less than that length, it is called mini-batch.
  <p>
This is how it works. Let's say the batch size is set to H, half the length of the entire feature set. We do normal back propogation of each sample. We compute the changes to be made to each weight. But now we keep a running total for each weight. We don't actually change the weight until we are done with H samples. Once we have seen H samples, we then update each weight with its running total divided by H.
    <p>
      At this point we have seen the first H samples. Now we get setup to take the next H samples and do the same.
      <p>
        Once we have run through all samples in the feature set, we are done with one epoch.
        <p>
The extremes of batch learning are (a) batch = full feature set, i.e., accumulate weight changes until very end then do just one weight adjustment, and (b) batch size = 1, i.e., what we have been doing. Between the extremes is mini-batch where batch size is less than full feature set and greater than 1.
<p>
I'd like you to write a new function, ann_simple_batch, that has this header:
  <pre>
 def ann_simple_batch(all_samples, labels, weights, bias, hypers={}):
  
  '''
  Can build an ANN with n input nodes and one output node.
  Uses sigmoid and mse.
  '''
  
  input_n = all_samples.shape[1]  #number of inputs in each sample
  
  assert weights.shape == (input_n,1), 'weights needs to have same shape as sample'
  assert all_samples.shape[0] >= 1, 'all_samples must represent 1 or more samples'
  assert bias.shape == (1,) , 'a single bias weight for output node'
  assert labels.shape[1] == 1, 'actual value for the 1 output node'
  assert labels.shape[0] == all_samples.shape[0], 'labels must match up with samples'
  
  hyper_keys = [*hypers]  #fails on 2.7
  target_set = set(['epochs', 'cost-reporting', 'learning-rate', 'batch-size'])  #might add more later
  diff_set = set(hyper_keys) - target_set
  if diff_set: print('WARNING: unrecognized hyper parameters ' + str(diff_set))

  max_epochs = hypers['epochs'] if 'epochs' in hypers else 100
  cost_reporting = hypers['cost-reporting'] if 'cost-reporting' in hypers else 100  #how often to report epoch cost
  alpha = hypers['learning-rate'] if 'learning-rate' in hypers else .05  #learning rate
batch_n = min(hypers['batch-size'],len(all_samples))  if 'batch-size' in hypers else 1  #batch size defaults to 1
  </pre>

In [0]:
def ann_simple_batch(all_samples, labels, weights, bias, hypers={}):
  
  '''
  Can build an ANN with n input nodes and one output node.
  Uses sigmoid and mse.
  '''
  
  input_n = all_samples.shape[1]  #number of inputs in each sample
  
  assert weights.shape == (input_n,1), 'weights needs to have same shape as sample'
  assert all_samples.shape[0] >= 1, 'all_samples must represent 1 or more samples'
  assert bias.shape == (1,) , 'a single bias weight for output node'
  assert labels.shape[1] == 1, 'actual value for the 1 output node'
  assert labels.shape[0] == all_samples.shape[0], 'labels must match up with samples'
  
  hyper_keys = [*hypers]  #fails on 2.7
  target_set = set(['epochs', 'cost-reporting', 'learning-rate', 'batch-size'])  #might add more later
  diff_set = set(hyper_keys) - target_set
  if diff_set: print('WARNING: unrecognized hyper parameters ' + str(diff_set))

  max_epochs = hypers['epochs'] if 'epochs' in hypers else 100
  cost_reporting = hypers['cost-reporting'] if 'cost-reporting' in hypers else 100  #how often to report epoch cost
  alpha = hypers['learning-rate'] if 'learning-rate' in hypers else .05  #learning rate
  batch_n = min(hypers['batch-size'],len(all_samples))  if 'batch-size' in hypers else 1  #batch size
   
  

In [0]:
#I'll give you this one - it calls your new batch function

def from_scratch_batch(samples, labels, hypers):
  
  input_n = samples.shape[1]
  
  #reset weights to initial values. Seed of 42 guarantees same random values
  np.random.seed(42)
  weights = np.random.rand(input_n,1)
  bias = np.random.rand(1)
  
  return ann_simple_batch(samples, labels, weights, bias, hypers)

In [38]:
#full batch on 3 columns

from_scratch_batch(feature_set, labels, hypers={'epochs':10000, 'cost-reporting':1000, 'learning-rate': .05, 'batch-size': len(feature_set)})

(0, array([0.225304]))
(1000, array([0.18963567]))
(2000, array([0.16724582]))
(3000, array([0.15781819]))
(4000, array([0.15358867]))
(5000, array([0.15142268]))
(6000, array([0.15018026]))
(7000, array([0.14940357]))
(8000, array([0.14888567]))
(9000, array([0.14852302]))
(10000, array([0.1482594]))


(array([[3.08764862],
        [0.44288226],
        [0.63812674]]), array([-2.11309206]))

Roughly the same weights as challenge 1. Let's check out accuracy.

In [39]:
#trying to meet or beat the results from challenge 1: 0.8146167557932263

ann_tester(feature_set, labels, [[3.08764862], [0.44288226], [0.63812674]], [-2.11309206])

0.8146167557932263

No change in accuracy. As reminder, with full batch we only update the weights every epoch. In our case we average one weight change over 561 samples. Even though are results are same as with challenge 1, here is more on why full-batch may not be the best approach: https://www.researchgate.net/post/How_to_update_weights_in_Batch_update_method_of_backpropagation. also see this https://datascience.stackexchange.com/questions/16807/why-mini-batch-size-is-better-than-one-single-batch-with-all-training-data
<p>
  Let's try batch size of 1. This is same as in challenge 1 so should get same results.

In [40]:
#batch size of 1 - should be same as seen in challenge 1: (array([[ 4.16472838], [ 0.57575141], [-0.74925534]]), array([-2.88134978]))

from_scratch_batch(feature_set, labels, hypers={'epochs':10000, 'cost-reporting':1000, 'learning-rate': .05, 'batch-size': 1})

(0, array([0.20333739]))
(1000, array([0.15038711]))
(2000, array([0.15023848]))
(3000, array([0.1502394]))
(4000, array([0.15023943]))
(5000, array([0.15023943]))
(6000, array([0.15023943]))
(7000, array([0.15023943]))
(8000, array([0.15023943]))
(9000, array([0.15023943]))
(10000, array([0.15023943]))


(array([[ 4.16472838],
        [ 0.57575141],
        [-0.74925534]]), array([-2.88134978]))

Same weights, as we expected.

<h3>Let's try mini-batch</h3>

I am using 32 because that is the current common wisdom in the deep learning community. This paper discusses, more generally, good values to choose for deep learning hyper-parameters: https://www.dropbox.com/s/h1wo6gajtlihdnw/setting_hypers.pdf?dl=0.

In [41]:
from_scratch_batch(feature_set, labels, hypers={'epochs':10000, 'cost-reporting':1000, 'learning-rate': .05, 'batch-size': 32})

(0, array([0.21895407]))
(1000, array([0.1526527]))
(2000, array([0.1473684]))
(3000, array([0.14726517]))
(4000, array([0.14723889]))
(5000, array([0.14722525]))
(6000, array([0.14721542]))
(7000, array([0.14720749]))
(8000, array([0.14720086]))
(9000, array([0.14719526]))
(10000, array([0.14719053]))


(array([[3.95638668],
        [0.5550847 ],
        [0.05681819]]), array([-2.92605609]))

In [42]:
ann_tester(feature_set, labels, [[3.95638668], [0.5550847 ], [0.05681819]], [-2.92605609])

0.8146167557932263

Same as with batch size = 1. In general, the larger the batch size the better. Why? Less weight updating. For the size of our problem it will be a negligible savings.
<p>
  But batch size has another important effect. It is a bit like the learning rate tamping down large swings of direction. Maybe you can guess why. In essence we are averaging weight changes over a batch. So it tends to smooth things out. I'll leave you with a pointer if you are interested in following up: https://stats.stackexchange.com/a/183990.

<h1>Extra Credit: </h2>

Try out `from_scratch_batch` on the Titanic data. See if you can beat the best score from the decision tree we built in week 4.
<p>
  I would include as many columns as you can. You can use Age directly - no binning necessary. Might be better to use ohe version of Pclass.
  <p>
    Remeber that `from_scratch_batch` will "stretch" the input weights to match your sample size. So I don't think you have to change any code. I think the main challenge is just wrangling `feature_set` and `labels` to get in shape for training.

In [0]:
#I'll give you a wrangled titanic table to get started with.

titanic_table = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vR2XbLnUuYlxaFMdS8_bBX3iKUIDEii6Lg5Rxesf-Oh8a6z8-vAN6UDGejaOrBg5130h4O_dLkecKWQ/pub?output=csv')
titanic_table.head(1)

In [0]:
fs.shape

In [0]:
lab.shape

In [0]:
#I just used ['Age', 'Sex', 'Pclass'] as input (i.e., 3 nodes). Bet you can do better with more columns.

from_scratch_batch(fs, lab, hypers={'epochs':10000, 'cost-reporting':1000, 'learning-rate': .05, 'batch-size': 32})

<hr>
<h1>Write it out</h1>
<div class=h1_cell>

You defined some new functions so add them to  `library_w19_deep_1b.py`.
<p>

</div>

In [0]:

output_array = (feature_set.tolist(), labels.tolist())
import pickle

with open('/content/gdrive/My Drive/class_tables/loan_labels_ann', 'wb') as fp:
    pickle.dump(output_array, fp)

In [0]:
with open ('/content/gdrive/My Drive/class_tables/loan_labels_ann', 'rb') as fp:
    test_out = pickle.load(fp)

In [0]:
fset = test_out[0]
labs = test_out[1]

In [0]:
nfset = np.array(fset)
(nfset == feature_set).all()

In [0]:
nlabs = np.array(labs)
(nlabs == labels).all()

In [0]:
print(nfset.shape)
print(nlabs.shape)

In [0]:
#should get (900, array([0.20541425]))

from_scratch(nfset, nlabs, hypers={'epochs':1000, 'cost-reporting':100, 'learning-rate': .05})

In [0]:
with open('/content/gdrive/My Drive/class_tables/loan_labels_ann.txt', 'w') as f:
  np.savetxt(f, labels)

In [0]:
with open('/content/gdrive/My Drive/class_tables/loan_labels_ann.txt', 'r') as f:
  check_labs = np.loadtxt(f)
(check_labs.shape, labels.shape)  #have to reestablish 2nd index of 1 - it is lost on writing

In [0]:
with open('/content/gdrive/My Drive/class_tables/loan_input_ann.txt', 'w') as f:
  np.savetxt(f, feature_set)

In [0]:
with open('/content/gdrive/My Drive/class_tables/loan_input_ann.txt', 'r') as f:
  check_input = np.loadtxt(f)
(check_input.shape,feature_set.shape)

In [0]:
feature_set.shape